<a href="https://colab.research.google.com/github/ManojKumar2105/Langchain-Task/blob/main/Output_Parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install langchain openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.5 MB/s eta 0:00:00


In [8]:
import os
import openai
import tiktoken
from getpass import getpass

OPENAI_API_KEY = getpass()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
openai.api_key = os.getenv("OPENAI_API_KEY")

··········


In [9]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0)

In [29]:
template_string_1= """You are a master branding consulatant who specializes in naming brands. \
You come up with catchy and memorable brand names.

Take the brand description below delimited by triple backticks and use it to create the name for a brand.

brand description: ```{brand_description}```

then based on the description and give hot new brand name give the brand a score 1-10 for how likely it is to succeed and give the reason for choosing the name.
"""

In [30]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string_1)

In [6]:
brand = prompt_template.format_messages( brand_description="Revolutionize your mobile experience with the sleek elegance and cutting-edge technology, where innovation meets style.")
brand

[HumanMessage(content='You are a master branding consulatant who specializes in naming brands. You come up with catchy and memorable brand names.\n\nTake the brand description below delimited by triple backticks and use it to create the name for a brand.\n\nbrand description: ```Revolutionize your mobile experience with the sleek elegance and cutting-edge technology, where innovation meets style.```\n\nthen based on the description and give hot new brand name give the brand a score 1-10 for how likely it is to succeed and give the reason for choosing the name.\n', additional_kwargs={}, example=False)]

In [31]:
response = llm(brand)
response

AIMessage(content='Brand Name: ElegaTech\n\nScore: 9\n\nReasoning: ElegaTech combines the words "elegance" and "technology" to create a catchy and memorable brand name. It reflects the sleek and stylish nature of the mobile experience being offered, while also emphasizing the cutting-edge innovation. The name has a modern and sophisticated feel, which aligns perfectly with the brand description. It is likely to succeed as it effectively communicates the brand\'s key attributes and creates a strong brand identity.', additional_kwargs={}, example=False)

In [11]:
print(response.content)

Brand Name: ElegaTech

Score: 9

Reasoning: ElegaTech captures the essence of the brand description by combining the words "elegance" and "technology." The name suggests a sophisticated and stylish mobile experience, aligning with the brand's promise of sleek elegance and cutting-edge technology. It is catchy, memorable, and evokes a sense of innovation and modernity. The name ElegaTech is likely to succeed as it effectively communicates the brand's unique selling points and resonates with the target audience.


#                **Main Methods Of Output Parser**

get_format_instructions() -> str: This method returns a string with instructions on how the LLM should structure the response.

parse(str) -> Any: This method accepts a string (usually the model's response)and returns it structured according to the instructions.

# **STRUCTURED OUTPUT PARSER**

In [12]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

brand_name_schema = ResponseSchema(
                        name="brand_name",
                        description="This is the name of the brand"
                    )

likelihood_of_success_schema = ResponseSchema(
                                  name="likelihood_of_success",
                                  description="This is an integer score between 1-10"
                              )

reasoning_schema = ResponseSchema(
                          name="reasoning",
                          description="This is the reasons for the score"
                    )

response_schemas = [brand_name_schema,
                    likelihood_of_success_schema,
                    reasoning_schema]

In [13]:
Structured_output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = Structured_output_parser.get_format_instructions()

format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"brand_name": string  // This is the name of the brand\n\t"likelihood_of_success": string  // This is an integer score between 1-10\n\t"reasoning": string  // This is the reasons for the score\n}\n```'

In [14]:
template_string = """You are a master branding consulatant who specializes in naming brands. \
You come up with catchy and memorable brand names.

Take the brand description below delimited by triple backticks and use it to create the name for a brand.

brand description: ```{brand_description}```

then based on the description and give hot new brand name give the brand a score 1-10 for how likely it is to succeed and give the reason for choosing the name.

{format_instructions}
"""

In [18]:
prompt = ChatPromptTemplate.from_template(template=template_string)
SOPmessages = prompt.format_messages(
                        brand_description="Revolutionize your mobile experience with the sleek elegance and cutting-edge technology, where innovation meets style.",
                        format_instructions=format_instructions
                        )

In [19]:
print(SOPmessages[0].content)

You are a master branding consulatant who specializes in naming brands. You come up with catchy and memorable brand names.

Take the brand description below delimited by triple backticks and use it to create the name for a brand.

brand description: ```Revolutionize your mobile experience with the sleek elegance and cutting-edge technology, where innovation meets style.```

then based on the description and give hot new brand name give the brand a score 1-10 for how likely it is to succeed and give the reason for choosing the name.

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"brand_name": string  // This is the name of the brand
	"likelihood_of_success": string  // This is an integer score between 1-10
	"reasoning": string  // This is the reasons for the score
}
```



In [21]:
SOPresponse = llm(SOPmessages)
SOPresponse

AIMessage(content='```json\n{\n\t"brand_name": "InnovaStyle",\n\t"likelihood_of_success": "9",\n\t"reasoning": "The name \'InnovaStyle\' combines the words \'innovation\' and \'style\' to reflect the brand\'s focus on cutting-edge technology and sleek elegance. It is catchy, memorable, and suggests a brand that is modern and forward-thinking. The name has a high likelihood of success as it effectively communicates the brand\'s key attributes and resonates with the target audience."\n}\n```', additional_kwargs={}, example=False)

In [24]:
SOPresponsemsg = Structured_output_parser.parse(SOPresponse.content)
SOPresponsemsg

{'brand_name': 'InnovaStyle',
 'likelihood_of_success': '9',
 'reasoning': "The name 'InnovaStyle' combines the words 'innovation' and 'style' to reflect the brand's focus on cutting-edge technology and sleek elegance. It is catchy, memorable, and suggests a brand that is modern and forward-thinking. The name has a high likelihood of success as it effectively communicates the brand's key attributes and resonates with the target audience."}

In [25]:
type(SOPresponsemsg["likelihood_of_success"])

str

# **Pydantic Output Parser**

In [26]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

In [27]:
class BrandInfo(BaseModel):
    brand_name: str = Field(description="This is the name of the brand")
    likelihood_of_success: int = Field(description="This is an integer score between 1-10")
    reasoning: str = Field(description="This is the reasons for the score")

    @validator('likelihood_of_success')
    def check_score(cls, field):
        if field >10:
            raise ValueError("Badly formed Score")
        return field

In [28]:
pydantic_parser = PydanticOutputParser(pydantic_object=BrandInfo)

pydantic_format_instructions = pydantic_parser.get_format_instructions()
print(pydantic_format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"brand_name": {"title": "Brand Name", "description": "This is the name of the brand", "type": "string"}, "likelihood_of_success": {"title": "Likelihood Of Success", "description": "This is an integer score between 1-10", "type": "integer"}, "reasoning": {"title": "Reasoning", "description": "This is the reasons for the score", "type": "string"}}, "required": ["brand_name", "likelihood_of_success", "reasoning"]}
```


In [35]:
prompt = ChatPromptTemplate.from_template(template=template_string)

pydantic_messages = prompt.format_messages(brand_description="Revolutionize your mobile experience with the sleek elegance and cutting-edge technology, where innovation meets style.",
                                format_instructions=pydantic_format_instructions)

In [38]:
pydantic_response = llm(pydantic_messages)
pydantic_response

AIMessage(content='{"brand_name": "InnovaStyle", "likelihood_of_success": 9, "reasoning": "The name combines the words \'innovation\' and \'style\', which perfectly captures the brand\'s promise of revolutionizing the mobile experience with sleek elegance and cutting-edge technology. It is catchy, memorable, and conveys a sense of modernity and sophistication. The name has a strong potential to resonate with the target audience and differentiate the brand from competitors."}', additional_kwargs={}, example=False)

In [39]:
pydantic_response_msg = pydantic_parser.parse(pydantic_response.content)
pydantic_response_msg

BrandInfo(brand_name='InnovaStyle', likelihood_of_success=9, reasoning="The name combines the words 'innovation' and 'style', which perfectly captures the brand's promise of revolutionizing the mobile experience with sleek elegance and cutting-edge technology. It is catchy, memorable, and conveys a sense of modernity and sophistication. The name has a strong potential to resonate with the target audience and differentiate the brand from competitors.")

In [40]:
type(pydantic_response_msg.likelihood_of_success)

int

# **List Parser**

In [41]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

List_parser = CommaSeparatedListOutputParser()
format_instructions = List_parser.get_format_instructions()
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz`'

In [79]:
List_template_string="Give me 4 possible marketing slogans for a mobile brand named {brand_name} :\n\n{format_instructions}"

List_prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(List_template_string)
    ],
    input_variables=["brand_name"],
    partial_variables={"format_instructions": format_instructions}
)


In [80]:
List_message = List_prompt.format_messages(brand_name=pydantic_response_msg.brand_name,
                                format_instructions=format_instructions)

In [81]:
List_response = llm(List_message)
slogans = List_parser.parse(List_response.content)
slogans

['"Innovate your style with InnovaStyle"\n"Unleash your mobile experience with InnovaStyle"\n"Stay ahead',
 'choose InnovaStyle"\n"Experience innovation',
 'choose InnovaStyle"']

# **Output_Fixing_Parser**

In [49]:
class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")

actor_query = "Generate the filmography for a random actor."

parser = PydanticOutputParser(pydantic_object=Actor)

In [52]:
Right_format = '{"name": "Chris Evans", "film_names": ["Captain America: The First Avenger"]}'

In [54]:
parser.parse(Right_format)

Actor(name='Chris Evans', film_names=['Captain America: The First Avenger'])

In [55]:
Wrong_Format ="{'name': 'Chris Evans', 'film_names': ['Captain America: The First Avenger']}"
parser.parse(Wrong_Format)

OutputParserException: ignored

In [56]:
from langchain.output_parsers import OutputFixingParser

new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())

In [57]:
new_parser.parse(Wrong_Format)

Actor(name='Chris Evans', film_names=['Captain America: The First Avenger'])

# **RetryOutputParser**

In [58]:
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser, RetryOutputParser

In [66]:
template = """Based on the user question, provide an Action and Action Input for what step should be taken.
{format_instructions}
Question: {query}
Response:"""
class Action(BaseModel):

    action: str = Field(description="action to take")
    action_input: str = Field(description="input to the action")

pyparser = PydanticOutputParser(pydantic_object=Action)

In [82]:
Fix_prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": pyparser.get_format_instructions()}
)

In [83]:
query = "who is Alan Turing?"
prompt_value = Fix_prompt.format_prompt(query=query)
prompt_value

StringPromptValue(text='Answer the user query.\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"action": {"title": "Action", "description": "action to take", "type": "string"}, "action_input": {"title": "Action Input", "description": "input to the action", "type": "string"}}, "required": ["action", "action_input"]}\n```\nwho is Alan Turing?\n')

In [71]:
# bad_response = '{"action": "search","action_input":"'+query+'"}'
bad_response = '{"action": "search"}'
pyparser.parse(bad_response)

OutputParserException: ignored

In [73]:
output_fix_parser = OutputFixingParser.from_llm(parser=pyparser, llm=ChatOpenAI())
output_fix_parser.parse(bad_response)

Action(action='search', action_input='keyword')

In [74]:
from langchain.output_parsers import RetryWithErrorOutputParser
retry_parser = RetryWithErrorOutputParser.from_llm(
    parser=pyparser, llm=OpenAI(temperature=0)
)

In [75]:
retry_parser.parse_with_prompt(bad_response, prompt_value)


Action(action='search', action_input='Alan Turing')

# **Datetime parser**


In [76]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import DatetimeOutputParser
from langchain.chains import LLMChain
from langchain.llms import OpenAI

In [77]:
Datetime = DatetimeOutputParser()
template = """Answer the users question:

{question}

{format_instructions}"""
prompt = PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": Datetime.get_format_instructions()},
)

In [78]:
Datetime.get_format_instructions()

'Write a datetime string that matches the \n            following pattern: "%Y-%m-%dT%H:%M:%S.%fZ". Examples: 1595-11-18T04:16:09.209251Z, 125-04-12T20:07:32.274374Z, 75-09-27T13:45:11.604056Z'

In [87]:
chain = LLMChain(prompt=prompt, llm=OpenAI())
chain.run("when did alan turing invent the turing machine?")

'\n\n1936-06-23T00:00:00.000000Z'